In [1]:
# best working till now 23-09-24

import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import time
import plotly.graph_objects as go
from tensorflow.keras.models import load_model
from IPython.display import display, clear_output

# Load your trained model for 5-minute future prediction
model_5min = load_model('model_5min.h5')

# Define the ticker symbol
ticker = "^NSEBANK"

# Fetch historical Bank Nifty data (last 60 minutes of data in 5-minute intervals)
historical_data = yf.download(ticker, period='1d', interval='5m')

# Preprocess the historical data
if not historical_data.empty:
    historical_data = historical_data[['Open', 'High', 'Low', 'Close']]
    scaler = MinMaxScaler(feature_range=(0, 1))

    # Fit the scaler only on historical data
    scaled_historical_data = scaler.fit_transform(historical_data)
else:
    print("No historical data found!")

# Initialize empty lists to store timestamps and candlestick data
timestamps = list(historical_data.index[-12:])  # Initialize with last 12 (5-minute) data points (60 minutes of historical timestamps)
open_prices = list(historical_data['Open'][-12:])  # Initialize with last 12 (5-minute) Open prices
high_prices = list(historical_data['High'][-12:])  # Initialize with last 12 (5-minute) High prices
low_prices = list(historical_data['Low'][-12:])    # Initialize with last 12 (5-minute) Low prices
actual_closes = list(historical_data['Close'][-12:])  # Initialize with the last actual 12 close prices
predicted_closes = []  # List to store predicted close prices
predicted_timestamps = []  # List to store timestamps for predicted future close prices

# Function for live prediction
def predict_live_data():
    live_data_accumulated = historical_data[-12:]  # Start with the last 60 minutes of historical data (12 intervals of 5 minutes)

    while True:
        # Fetch live data (5-minute interval)
        live_data = yf.download(ticker, period='1d', interval='5m')

        # Check if live data is empty
        if live_data.empty:
            print("No live data found! Skipping this cycle.")
            time.sleep(300)  # Wait 5 minutes and try again
            continue

        # Preprocess live data
        live_data = live_data[['Open', 'High', 'Low', 'Close']]

        # Append new live data to accumulated data and keep only the last 60 minutes (12 data points)
        live_data_accumulated = pd.concat([live_data_accumulated, live_data])[-12:]

        # Apply the scaler to the live data
        try:
            scaled_live_data = scaler.transform(live_data_accumulated)
        except ValueError as e:
            print(f"Error scaling live data: {e}")
            time.sleep(300)  # Wait 5 minutes and try again
            continue

        # Ensure there's enough data for prediction (12 data points of 5-minute intervals)
        if len(scaled_live_data) < 12:
            print("Not enough data for prediction! Waiting for more data.")
            time.sleep(300)
            continue

        # The model should now predict 5 minutes into the future
        X_live = np.array([scaled_live_data[-12:]])

        # Reshape to match the LSTM model's input shape
        X_live = np.reshape(X_live, (X_live.shape[0], X_live.shape[1], 4))

        # Make prediction for 5 minutes into the future
        predicted_price = model_5min.predict(X_live)

        # Inverse transform to get the actual predicted price (predicted 5-min ahead close price)
        predicted_close_price = scaler.inverse_transform([[0, 0, 0, predicted_price[0][0]]])[0][3]

        # Get the current timestamp and other OHLC values from live data
        current_time = live_data.index[-1]
        open_price = live_data['Open'][-1]
        high_price = live_data['High'][-1]
        low_price = live_data['Low'][-1]
        actual_close_price = live_data['Close'][-1]  # Actual close price

        # Store predicted values for 5 minutes in the future
        predicted_timestamps.append(current_time + pd.Timedelta(minutes=5))  # Add 5 minutes to current time
        predicted_closes.append(predicted_close_price)

        # Append the OHLC values and actual close price to the lists
        timestamps.append(current_time)
        open_prices.append(open_price)
        high_prices.append(high_price)
        low_prices.append(low_price)
        actual_closes.append(actual_close_price)

        # Output predicted price
        print(f"Predicted Close Price for 5 minutes ahead: {predicted_close_price}")
        print(f"Actual Close Price: {actual_close_price}")

        # Create a new candlestick figure
        fig = go.Figure()

        # Add candlestick trace with real open, high, low, and actual close prices
        fig.add_trace(go.Candlestick(x=timestamps,
                                     open=open_prices,
                                     high=high_prices,
                                     low=low_prices,
                                     close=actual_closes,
                                     name='Actual Candlestick'))

        # Overlay predicted candlesticks using predicted close prices
        # We replicate open, high, low using the predicted close (since we don't have future OHLC values)
        fig.add_trace(go.Candlestick(x=predicted_timestamps,
                                     open=predicted_closes,  # Use the predicted close for open, high, low
                                     high=predicted_closes,
                                     low=predicted_closes,
                                     close=predicted_closes,
                                     name='Predicted Candlestick',
                                     increasing_line_color='blue', decreasing_line_color='blue'))

        # Update layout with hovermode for both x and y axes
        fig.update_layout(
            title="Live 5-Minute Interval Candlestick with Prediction (5 Min Ahead)",
            xaxis_title="Timestamp",
            yaxis_title="Price",
            hovermode="x unified",  # Unified hover mode on x-axis
            hoverlabel=dict(namelength=-1),  # Show full text
        )

        # Clear the previous plot and display the updated candlestick chart
        clear_output(wait=True)
        display(fig)

        # Wait for 5 minutes before running the prediction again
        time.sleep(60)

# Start the live prediction and graph update
predict_live_data()


ValueError: Unrecognized keyword arguments: ['batch_shape']

In [2]:
model_5min = load_model('model_5min.h5', compile=False)
model_5min.summary()


ValueError: Unrecognized keyword arguments: ['batch_shape']

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import time
import plotly.graph_objects as go
from tensorflow.keras.models import load_model
from IPython.display import display, clear_output

# Load your trained model for 5-minute future prediction
model_5min = load_model('model_5min.h5')

# Define the ticker symbol
ticker = "^NSEBANK"

# Fetch historical Bank Nifty data (last 60 minutes of data in 5-minute intervals)
historical_data = yf.download(ticker, period='1d', interval='5m')

# Preprocess the historical data
if not historical_data.empty:
    historical_data = historical_data[['Open', 'High', 'Low', 'Close']]
    scaler = MinMaxScaler(feature_range=(0, 1))

    # Fit the scaler only on historical data
    scaled_historical_data = scaler.fit_transform(historical_data)
else:
    print("No historical data found!")

# Initialize empty lists to store timestamps and candlestick data
timestamps = list(historical_data.index[-12:])  # Initialize with last 12 (5-minute) data points (60 minutes of historical timestamps)
open_prices = list(historical_data['Open'][-12:])  # Initialize with last 12 (5-minute) Open prices
high_prices = list(historical_data['High'][-12:])  # Initialize with last 12 (5-minute) High prices
low_prices = list(historical_data['Low'][-12:])    # Initialize with last 12 (5-minute) Low prices
predicted_closes = []  # List to store predicted close prices
actual_closes = list(historical_data['Close'][-12:])  # Initialize with the last actual 12 close prices

# Function for live prediction
def predict_live_data():
    live_data_accumulated = historical_data[-12:]  # Start with the last 60 minutes of historical data (12 intervals of 5 minutes)

    while True:
        # Fetch live data (5-minute interval)
        live_data = yf.download(ticker, period='1d', interval='1m')

        # Check if live data is empty
        if live_data.empty:
            print("No live data found! Skipping this cycle.")
            time.sleep(300)  # Wait 5 minutes and try again
            continue

        # Preprocess live data
        live_data = live_data[['Open', 'High', 'Low', 'Close']]

        # Append new live data to accumulated data and keep only the last 60 minutes (12 data points)
        live_data_accumulated = pd.concat([live_data_accumulated, live_data])[-12:]

        # Apply the scaler to the live data
        try:
            scaled_live_data = scaler.transform(live_data_accumulated)
        except ValueError as e:
            print(f"Error scaling live data: {e}")
            time.sleep(300)  # Wait 5 minutes and try again
            continue

        # Ensure there's enough data for prediction (12 data points of 5-minute intervals)
        if len(scaled_live_data) < 12:
            print("Not enough data for prediction! Waiting for more data.")
            time.sleep(300)
            continue

        # The model should now predict 5 minutes into the future
        X_live = np.array([scaled_live_data[-12:]])

        # Reshape to match the LSTM model's input shape
        X_live = np.reshape(X_live, (X_live.shape[0], X_live.shape[1], 4))

        # Make prediction for 5 minutes into the future
        predicted_price = model_5min.predict(X_live)

        # Inverse transform to get the actual predicted price (predicted 5-min ahead close price)
        predicted_close_price = scaler.inverse_transform([[0, 0, 0, predicted_price[0][0]]])[0][3]

        # Get the current timestamp and other OHLC values from live data
        current_time = live_data.index[-1]
        open_price = live_data['Open'][-1]
        high_price = live_data['High'][-1]
        low_price = live_data['Low'][-1]
        actual_close_price = live_data['Close'][-1]  # Actual close price

        # Append the OHLC values and predicted close price to the lists
        timestamps.append(current_time)
        open_prices.append(open_price)
        high_prices.append(high_price)
        low_prices.append(low_price)
        predicted_closes.append(predicted_close_price)
        actual_closes.append(actual_close_price)  # Append the actual close price as well

        # Add the predicted price to the live data to simulate prediction of future 5-minute close
        new_row = pd.DataFrame([[open_price, high_price, low_price, predicted_close_price]],
                               columns=['Open', 'High', 'Low', 'Close'], index=[current_time])
        live_data_accumulated = pd.concat([live_data_accumulated, new_row])[-12:]

        # Output predicted price
        print(f"Predicted Close Price for 5 minutes ahead: {predicted_close_price}")
        print(f"Actual Close Price: {actual_close_price}")

        # Create a new candlestick figure
        fig = go.Figure()

        # Add candlestick trace with real open, high, low, and actual close prices
        fig.add_trace(go.Candlestick(x=timestamps,
                                     open=open_prices,
                                     high=high_prices,
                                     low=low_prices,
                                     close=actual_closes,
                                     name='Actual Candlestick'))

        # Overlay predicted close prices as a separate line
        fig.add_trace(go.Scatter(x=timestamps, 
                                 y=predicted_closes, 
                                 mode='lines+markers', 
                                 name='Predicted Close', 
                                 line=dict(color='blue', dash='dash')))

        # Update layout with hovermode for both x and y axes
        fig.update_layout(
            title="Live 5-Minute Interval Candlestick with Prediction (5 Min Ahead)",
            xaxis_title="Timestamp",
            yaxis_title="Price",
            hovermode="x unified",  # Unified hover mode on x-axis
            hoverlabel=dict(namelength=-1),  # Show full text
        )

        # Clear the previous plot and display the updated candlestick chart
        clear_output(wait=True)
        display(fig)

        # Wait for 5 minutes before running the prediction again
        time.sleep(5)

# Start the live prediction and graph update
predict_live_data()


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import time
import plotly.graph_objects as go
from tensorflow.keras.models import load_model
from IPython.display import display, clear_output

# Load your trained model for 5-minute future prediction
model_5min = load_model('model_5min.h5')

# Define the ticker symbol
ticker = "^NSEBANK"

# Fetch historical Bank Nifty data (last 60 minutes of data in 5-minute intervals)
historical_data = yf.download(ticker, period='1d', interval='5m')

# Preprocess the historical data
if not historical_data.empty:
    historical_data = historical_data[['Open', 'High', 'Low', 'Close']]
    scaler = MinMaxScaler(feature_range=(0, 1))

    # Fit the scaler only on historical data
    scaled_historical_data = scaler.fit_transform(historical_data)
else:
    print("No historical data found!")

# Initialize empty lists to store timestamps and candlestick data
timestamps = list(historical_data.index[-12:])  # Initialize with last 12 (5-minute) data points (60 minutes of historical timestamps)
open_prices = list(historical_data['Open'][-12:])  # Initialize with last 12 (5-minute) Open prices
high_prices = list(historical_data['High'][-12:])  # Initialize with last 12 (5-minute) High prices
low_prices = list(historical_data['Low'][-12:])    # Initialize with last 12 (5-minute) Low prices
predicted_closes = []  # List to store predicted close prices

# Function for live prediction
def predict_live_data():
    live_data_accumulated = historical_data[-12:]  # Start with the last 60 minutes of historical data (12 intervals of 5 minutes)

    while True:
        # Fetch live data (5-minute interval)
        live_data = yf.download(ticker, period='1d', interval='5m')

        # Check if live data is empty
        if live_data.empty:
            print("No live data found! Skipping this cycle.")
            time.sleep(300)  # Wait 5 minutes and try again
            continue

        # Preprocess live data
        live_data = live_data[['Open', 'High', 'Low', 'Close']]

        # Append new live data to accumulated data and keep only the last 60 minutes (12 data points)
        live_data_accumulated = pd.concat([live_data_accumulated, live_data])[-12:]

        # Apply the scaler to the live data
        try:
            scaled_live_data = scaler.transform(live_data_accumulated)
        except ValueError as e:
            print(f"Error scaling live data: {e}")
            time.sleep(300)  # Wait 5 minutes and try again
            continue

        # Ensure there's enough data for prediction (12 data points of 5-minute intervals)
        if len(scaled_live_data) < 12:
            print("Not enough data for prediction! Waiting for more data.")
            time.sleep(300)
            continue

        # The model should now predict 5 minutes into the future
        X_live = np.array([scaled_live_data[-12:]])

        # Reshape to match the LSTM model's input shape
        X_live = np.reshape(X_live, (X_live.shape[0], X_live.shape[1], 4))

        # Make prediction for 5 minutes into the future
        predicted_price = model_5min.predict(X_live)

        # Inverse transform to get the actual predicted price (predicted 5-min ahead close price)
        predicted_close_price = scaler.inverse_transform([[0, 0, 0, predicted_price[0][0]]])[0][3]

        # Get the current timestamp and other OHLC values from live data
        current_time = live_data.index[-1]
        open_price = live_data['Open'][-1]
        high_price = live_data['High'][-1]
        low_price = live_data['Low'][-1]

        # Append the OHLC values and predicted close price to the lists
        timestamps.append(current_time)
        open_prices.append(open_price)
        high_prices.append(high_price)
        low_prices.append(low_price)
        predicted_closes.append(predicted_close_price)

        # Add the predicted price to the live data to simulate prediction of future 5-minute close
        new_row = pd.DataFrame([[open_price, high_price, low_price, predicted_close_price]],
                               columns=['Open', 'High', 'Low', 'Close'], index=[current_time])
        live_data_accumulated = pd.concat([live_data_accumulated, new_row])[-12:]

        # Output predicted price
        print(f"Predicted Close Price for 5 minutes ahead: {predicted_close_price}")

        # Create a new candlestick figure
        fig = go.Figure()

        # Add candlestick trace with real open, high, low, and predicted close prices
        fig.add_trace(go.Candlestick(x=timestamps,
                                     open=open_prices,
                                     high=high_prices,
                                     low=low_prices,
                                     close=predicted_closes,
                                     name='Candlestick'))

        # Update layout with hovermode for both x and y axes
        fig.update_layout(
            title="Live 5-Minute Interval Candlestick Prediction (5 Min Ahead)",
            xaxis_title="Timestamp",
            yaxis_title="Price",
            hovermode="x unified",  # Unified hover mode on x-axis
            hoverlabel=dict(namelength=-1),  # Show full text
        )

        # Clear the previous plot and display the updated candlestick chart
        clear_output(wait=True)
        display(fig)

        # Wait for 5 minutes before running the prediction again
        time.sleep(5)

# Start the live prediction and graph update
predict_live_data()
